Transactions with DB-API 2.0
============================

This is a short notebook demonstrating how transactions are handled in DB-API 2.0 .

In [1]:
# Remove any existing database created in previous runs of this notebook.
import os

if os.path.isfile('example.db'):
    os.unlink('example.db')

In [2]:
import sqlite3

# ":memory:" is sqlite-specific, and creates a database in memory
connection = sqlite3.connect(":memory:")

We will first have a look at manually creating, committing, and rolling back transactions.

In [3]:
# Create a simple key -> value table.
cursor = connection.cursor()
cursor.execute("CREATE TABLE example (id INT PRIMARY KEY, value VARCHAR(255))")

In [4]:
# Insert a single row, and explicitly commit the transaction to the DB.
cursor.execute("INSERT INTO example VALUES (1, 'hello')")
connection.commit()
result = cursor.execute("SELECT COUNT(*) FROM example").fetchone()
print "{} row(s)".format(result[0])

1 row(s)


In [5]:
# A `rollback` operation undoes all operations in a transaction.
cursor.execute("INSERT INTO example VALUES (2, 'bonjour')")
cursor.execute("INSERT INTO example VALUES (3, 'adieu')")
result = cursor.execute("SELECT COUNT(*) FROM example").fetchone()
print "{} row(s)".format(result[0])
connection.rollback()

# The two rows added in the previous transaction are not in the DB.
result = cursor.execute("SELECT COUNT(*) FROM example").fetchone()
print "{} row(s)".format(result[0])

3 row(s)
1 row(s)


In [6]:
cursor.close()
connection.close()

Now we will do similar operations, using the context manager to group
operations in transactions and automatically committing them if the
block exits without problems.

In [7]:
# The context manager wraps a code block in a single transaction.
uri = "example.db"
connection = sqlite3.connect(uri)
with connection:
    cursor = connection.cursor()
    cursor.execute("CREATE TABLE example (id INT, value VARCHAR(255))")
    cursor.execute("INSERT INTO example VALUES (1, 'hello')")
    cursor.execute("INSERT INTO example VALUES (2, 'auf Wiedersehen')")

# The transaction completed cleanly, so it was committed at the exit of
# the context manager.
result = cursor.execute("SELECT COUNT(*) FROM example").fetchone()
print "{} row(s)".format(result[0])

2 row(s)


In [8]:
# If something bad happens during the transaction, the context manager
# automatically rolls it back.
with connection:
    cursor = connection.cursor()
    cursor.execute("INSERT INTO example VALUES (3, 'bonjour')")
    raise ValueError("dummy exception")
    cursor.execute("INSERT INTO example VALUES (4, 'adieu')")

ValueError: dummy exception

In [9]:
# How many rows are in the table right now?
with connection:
    cursor = connection.cursor()
    print cursor.execute("SELECT * FROM example").fetchall()

# Note: some drivers support so-called autocommit 
# (every execute is effectively a transaction).
# In that case, a different result would be expected here.

[(1, u'hello'), (2, u'auf Wiedersehen')]


In [10]:
# Let's commit them 
with connection:
    cursor = connection.cursor()
    cursor.execute("INSERT INTO example VALUES (3, 'bonjour')")
    cursor.execute("INSERT INTO example VALUES (4, 'adieu')")
        
with connection:
    cursor = connection.cursor()
    print cursor.execute("SELECT * FROM example").fetchall()

[(1, u'hello'), (2, u'auf Wiedersehen'), (3, u'bonjour'), (4, u'adieu')]


In [11]:
connection.close()